In general, there are two methods for sending transactions using web3.py: `send_transaction` and `send_raw_transaction`. A brief guide:

1) Want to sign a transaction offline or send pre-signed transactions?
    - use `sign_transaction` + `send_raw_transaction`
1) Are you primarily using the same account for all transactions and would you prefer to save a few lines of code?
    - configure `construct_sign_and_send_raw_middleware`, then
    - use `send_transaction`
1) Otherwise:
    - load account via eth-account (`w3.eth.account.from_key(pk)`)
    - use `send_transaction`
    
Interacting with or deploying a contract?
- Option 1: `transact` uses `send_transaction` under the hood
- Option 2: `build_transaction` + `sign_transaction` + `send_raw_transaction`

(Examples below.)

### Setup

In [1]:
from pprint import pprint
from web3 import Web3, EthereumTesterProvider

w3 = Web3(EthereumTesterProvider())
print(f"web.py version: {w3.api}")
print(f"provider: {w3.client_version}")

web.py version: 6.1.0
provider: EthereumTester/0.8.0b3/darwin/python3.9.7


### Chapter 0: `w3.eth.send_transaction` (with `eth-tester`)

Many tutorials use `eth-tester` for convenience and speed of conveying ideas/building a proof of concept. Transactions sent by test accounts are auto-signed.

In [2]:
# eth-tester populates accounts with test ether:
print(w3.eth.accounts)

acct1 = w3.eth.accounts[0]
bal = w3.from_wei(w3.eth.get_balance(acct1), 'ether')
print(f"\nAccount at address {acct1} contains {bal} ether")

['0x7E5F4552091A69125d5DfCb7b8C2659029395Bdf', '0x2B5AD5c4795c026514f8317c7a215E218DcCD6cF', '0x6813Eb9362372EEF6200f3b1dbC3f819671cBA69', '0x1efF47bc3a10a45D4B230B5d10E37751FE6AA718', '0xe1AB8145F7E55DC933d51a18c793F901A3A0b276', '0xE57bFE9F44b819898F47BF37E5AF72a0783e1141', '0xd41c057fd1c78805AAC12B0A94a405c0461A6FBb', '0xF1F6619B38A98d6De0800F1DefC0a6399eB6d30C', '0xF7Edc8FA1eCc32967F827C9043FcAe6ba73afA5c', '0x4CCeBa2d7D2B4fdcE4304d3e09a1fea9fbEb1528']

Account at address 0x7E5F4552091A69125d5DfCb7b8C2659029395Bdf contains 1000000 ether


In [3]:
some_address = "0x0000000000000000000000000000000000000000"

# when using one of its generated test accounts,
# eth-tester signs the tx (under the hood) before sending:
tx_hash = w3.eth.send_transaction({
    "from": acct1,
    "to": some_address,
    "value": 123123123123123
})

tx = w3.eth.get_transaction(tx_hash)
assert tx["from"] == acct1

### Chapter 1: `w3.eth.send_transaction` + signer middleware

*Note: example utilizes a hard-coded private key. Never do this! Pull it from an environment variable instead.*

The `send_transaction` method is convenient and to-the-point. If you want to continue using the pattern after graduating from `eth-tester`, you can utilize web3.py middleware to sign transactions from a particular account:

In [4]:
from web3.middleware import construct_sign_and_send_raw_middleware

# Example private key:
pk = '0x7ec01898a720beda9304921aec62735846aa1a545a8332bc030577c21f72cd46'
#### Note: Never commit your key in your code! Use env variables instead:
#### import os
#### pk = os.environ.get('PRIVATE_KEY')

# Instantiate an Account object from your key:
acct2 = w3.eth.account.from_key(pk)

# For the sake of this example, fund the new account:
w3.eth.send_transaction({
    "from": acct1,
    "value": w3.to_wei(3, 'ether'),
    "to": acct2.address
})
print(f"acct2 balance: {w3.eth.get_balance(acct2.address)}")

# Add acct2 as auto-signer:
w3.middleware_onion.add(construct_sign_and_send_raw_middleware(acct2))
# pk also works: w3.middleware_onion.add(construct_sign_and_send_raw_middleware(pk))

# Transactions from `acct2` will then be signed, under the hood, in the middleware:
tx_hash = w3.eth.send_transaction({
    "from": acct2.address,
    "value": 3333333333,
    "to": some_address
})

tx = w3.eth.get_transaction(tx_hash)
assert tx["from"] == acct2.address

# Optionally, you can set a default signer as well:
# w3.eth.default_account = acct2.address
# Then, if you omit a "from" key, acct2 will be used.

acct2 balance: 3000000000000000000


### Chapter 2: `w3.eth.send_raw_transaction`

if you don't opt for the middleware, you'll need to: 
- build each transaction, 
- `sign_transaction`, and 
- then use `send_raw_transaction`.

In [5]:
# 1. Build a new tx
transaction = {
    'from': acct2.address,
    'to': some_address,
    'value': 1000000000,
    'nonce': w3.eth.get_transaction_count(acct2.address),
    'gas': 200000,
    'maxFeePerGas': 2000000000,
    'maxPriorityFeePerGas': 1000000000,
}

# 2. Sign tx with a private key
signed = w3.eth.account.sign_transaction(transaction, pk)

# 2.a. The "raw" transaction is what will be broadcast to the network:
print(f"Raw tx: {signed.rawTransaction.hex()}")

# 3. Send the signed transaction
tx_hash = w3.eth.send_raw_transaction(signed.rawTransaction)
tx = w3.eth.get_transaction(tx_hash)
assert tx["from"] == acct2.address

Raw tx: 0x02f86f8001843b9aca00847735940083030d40940000000000000000000000000000000000000000843b9aca0080c080a007e2f4dd64a56a6c7efc1b5c8c47bf7e2cf3572548dbe5c13e13e6f44c0590cfa058740b159934db22fd84a683edfd4c5a69022567d43c9a18347ad4abcd576d50


### Chapter 3: Contract transactions

The same concepts apply for contract interactions, at least under the hood.

Executing a function on a smart contract requires sending a transaction, which is typically done in one of two ways:
- executing the `transact` function, or
- `build_transaction`, then signing and sending the raw transaction.

In [6]:
#########################################
#### SMOL CONTRACT FOR THIS EXAMPLE: ####
#########################################
# // SPDX-License-Identifier: MIT
# pragma solidity 0.8.17;
#
# contract Billboard {
#     string public message;
#
#     constructor(string memory _message) {
#         message = _message;
#     }
#   
#     function writeBillboard(string memory _message) public {
#         message = _message;
#     }
# }
init_bytecode = "60806040523480156200001157600080fd5b5060405162000bee38038062000bee8339818101604052810190620000379190620001e3565b80600090816200004891906200047f565b505062000566565b6000604051905090565b600080fd5b600080fd5b600080fd5b600080fd5b6000601f19601f8301169050919050565b7f4e487b7100000000000000000000000000000000000000000000000000000000600052604160045260246000fd5b620000b9826200006e565b810181811067ffffffffffffffff82111715620000db57620000da6200007f565b5b80604052505050565b6000620000f062000050565b9050620000fe8282620000ae565b919050565b600067ffffffffffffffff8211156200012157620001206200007f565b5b6200012c826200006e565b9050602081019050919050565b60005b83811015620001595780820151818401526020810190506200013c565b60008484015250505050565b60006200017c620001768462000103565b620000e4565b9050828152602081018484840111156200019b576200019a62000069565b5b620001a884828562000139565b509392505050565b600082601f830112620001c857620001c762000064565b5b8151620001da84826020860162000165565b91505092915050565b600060208284031215620001fc57620001fb6200005a565b5b600082015167ffffffffffffffff8111156200021d576200021c6200005f565b5b6200022b84828501620001b0565b91505092915050565b600081519050919050565b7f4e487b7100000000000000000000000000000000000000000000000000000000600052602260045260246000fd5b600060028204905060018216806200028757607f821691505b6020821081036200029d576200029c6200023f565b5b50919050565b60008190508160005260206000209050919050565b60006020601f8301049050919050565b600082821b905092915050565b600060088302620003077fffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffff82620002c8565b620003138683620002c8565b95508019841693508086168417925050509392505050565b6000819050919050565b6000819050919050565b6000620003606200035a62000354846200032b565b62000335565b6200032b565b9050919050565b6000819050919050565b6200037c836200033f565b620003946200038b8262000367565b848454620002d5565b825550505050565b600090565b620003ab6200039c565b620003b881848462000371565b505050565b5b81811015620003e057620003d4600082620003a1565b600181019050620003be565b5050565b601f8211156200042f57620003f981620002a3565b6200040484620002b8565b8101602085101562000414578190505b6200042c6200042385620002b8565b830182620003bd565b50505b505050565b600082821c905092915050565b6000620004546000198460080262000434565b1980831691505092915050565b60006200046f838362000441565b9150826002028217905092915050565b6200048a8262000234565b67ffffffffffffffff811115620004a657620004a56200007f565b5b620004b282546200026e565b620004bf828285620003e4565b600060209050601f831160018114620004f75760008415620004e2578287015190505b620004ee858262000461565b8655506200055e565b601f1984166200050786620002a3565b60005b8281101562000531578489015182556001820191506020850194506020810190506200050a565b868310156200055157848901516200054d601f89168262000441565b8355505b6001600288020188555050505b505050505050565b61067880620005766000396000f3fe608060405234801561001057600080fd5b50600436106100365760003560e01c8063615362111461003b578063e21f37ce14610057575b600080fd5b61005560048036038101906100509190610270565b610075565b005b61005f610088565b60405161006c9190610338565b60405180910390f35b80600090816100849190610570565b5050565b6000805461009590610389565b80601f01602080910402602001604051908101604052809291908181526020018280546100c190610389565b801561010e5780601f106100e35761010080835404028352916020019161010e565b820191906000526020600020905b8154815290600101906020018083116100f157829003601f168201915b505050505081565b6000604051905090565b600080fd5b600080fd5b600080fd5b600080fd5b6000601f19601f8301169050919050565b7f4e487b7100000000000000000000000000000000000000000000000000000000600052604160045260246000fd5b61017d82610134565b810181811067ffffffffffffffff8211171561019c5761019b610145565b5b80604052505050565b60006101af610116565b90506101bb8282610174565b919050565b600067ffffffffffffffff8211156101db576101da610145565b5b6101e482610134565b9050602081019050919050565b82818337600083830152505050565b600061021361020e846101c0565b6101a5565b90508281526020810184848401111561022f5761022e61012f565b5b61023a8482856101f1565b509392505050565b600082601f8301126102575761025661012a565b5b8135610267848260208601610200565b91505092915050565b60006020828403121561028657610285610120565b5b600082013567ffffffffffffffff8111156102a4576102a3610125565b5b6102b084828501610242565b91505092915050565b600081519050919050565b600082825260208201905092915050565b60005b838110156102f35780820151818401526020810190506102d8565b60008484015250505050565b600061030a826102b9565b61031481856102c4565b93506103248185602086016102d5565b61032d81610134565b840191505092915050565b6000602082019050818103600083015261035281846102ff565b905092915050565b7f4e487b7100000000000000000000000000000000000000000000000000000000600052602260045260246000fd5b600060028204905060018216806103a157607f821691505b6020821081036103b4576103b361035a565b5b50919050565b60008190508160005260206000209050919050565b60006020601f8301049050919050565b600082821b905092915050565b60006008830261041c7fffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffff826103df565b61042686836103df565b95508019841693508086168417925050509392505050565b6000819050919050565b6000819050919050565b600061046d6104686104638461043e565b610448565b61043e565b9050919050565b6000819050919050565b61048783610452565b61049b61049382610474565b8484546103ec565b825550505050565b600090565b6104b06104a3565b6104bb81848461047e565b505050565b5b818110156104df576104d46000826104a8565b6001810190506104c1565b5050565b601f821115610524576104f5816103ba565b6104fe846103cf565b8101602085101561050d578190505b610521610519856103cf565b8301826104c0565b50505b505050565b600082821c905092915050565b600061054760001984600802610529565b1980831691505092915050565b60006105608383610536565b9150826002028217905092915050565b610579826102b9565b67ffffffffffffffff81111561059257610591610145565b5b61059c8254610389565b6105a78282856104e3565b600060209050601f8311600181146105da57600084156105c8578287015190505b6105d28582610554565b86555061063a565b601f1984166105e8866103ba565b60005b82811015610610578489015182556001820191506020850194506020810190506105eb565b8683101561062d5784890151610629601f891682610536565b8355505b6001600288020188555050505b50505050505056fea2646970667358221220888d8a12f4a4af25d9b35ebc274495e369542aca3b0433520a352a345624fbf564736f6c63430008110033"
abi = '[{"inputs": [{"internalType": "string","name": "_message","type": "string"}],"stateMutability": "nonpayable","type": "constructor"},{"inputs": [],"name": "message","outputs": [{"internalType": "string","name": "","type": "string"}],"stateMutability": "view","type": "function"},{"inputs": [{"internalType": "string","name": "_message","type": "string"}],"name": "writeBillboard","outputs": [], "stateMutability": "nonpayable", "type": "function"}]'

# Initialize the contract factory:
Billboard = w3.eth.contract(bytecode=init_bytecode, abi=abi)

# Deploy a contract using `transact` + the signer middleware:
tx_hash = Billboard.constructor("gm").transact({"from": acct2.address})
receipt = w3.eth.get_transaction_receipt(tx_hash)
deployed_addr = receipt["contractAddress"]
print(f"Billboard contract deployed at address: {deployed_addr}")

billboard = w3.eth.contract(address=deployed_addr, abi=abi)
billboard.all_functions()

Billboard contract deployed at address: 0xa37E9655e17378Cd47970e065dedD0eEF5FCE0ef


[<Function message()>, <Function writeBillboard(string)>]

In [7]:
# Manually build and sign a transaction:
unsent_billboard_tx = billboard.functions.writeBillboard("gn").build_transaction({
    "from": acct2.address,
    "nonce": w3.eth.get_transaction_count(acct2.address),
})
pprint(unsent_billboard_tx)

{'chainId': 131277322940537,
 'data': '0x6153621100000000000000000000000000000000000000000000000000000000000000200000000000000000000000000000000000000000000000000000000000000002676e000000000000000000000000000000000000000000000000000000000000',
 'from': '0x043414E7Ec0b1dDdD7d2a69b53c2bA498F3167cB',
 'gas': 36134,
 'maxFeePerGas': 2026637978,
 'maxPriorityFeePerGas': 1000000000,
 'nonce': 3,
 'to': '0xa37E9655e17378Cd47970e065dedD0eEF5FCE0ef',
 'value': 0}


In [8]:
signed_tx = w3.eth.account.sign_transaction(unsent_billboard_tx, private_key=pk)

assert billboard.functions.message().call() == "gm"

tx_hash = w3.eth.send_raw_transaction(signed_tx.rawTransaction)
w3.eth.wait_for_transaction_receipt(tx_hash)

assert billboard.functions.message().call() == "gn"